<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">CONTENT-BASED FILTERING</font></h1>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous, and can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore Content-based recommendation systems and implement a simple version of one using Python and the Pandas library.

### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Content-Based Filtering</a></li>
    </ol>
</div>
<br>

<a id="ref1"></a>
# Acquiring the Data

To acquire and extract the data, simply run the following Bash scripts:  
Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens/). Lets download the dataset. To download the data, we will use **`!wget`** to download it from IBM Object Storage.  
__Did you know?__ When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)

In [1]:
#!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
#!unzip -o -j moviedataset.zip 

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


Now you're ready to start working with the data!

<a id="ref2"></a>
# Preprocessing

First, let's get all of the imports out of the way:

In [2]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:

In [3]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Let's also remove the year from the __title__ column by using pandas' replace function and store in a new __year__ column.

In [4]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also split the values in the __Genres__ column into a __list of Genres__ to simplify future use. This can be achieved by applying Python's split string function on the correct column.

In [5]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.

In [14]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        #create a new coulumn for each new genre type in the dataframe and set its value with 1 
        # and if this genre has a coulumn set the a value only without creating a new one 
        moviesWithGenres_df.at[index, genre] = 1
        print(genre)
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

Adventure
Animation
Children
Comedy
Fantasy
Adventure
Children
Fantasy
Comedy
Romance
Comedy
Drama
Romance
Comedy
Action
Crime
Thriller
Comedy
Romance
Adventure
Children
Action
Action
Adventure
Thriller
Comedy
Drama
Romance
Comedy
Horror
Adventure
Animation
Children
Drama
Action
Adventure
Romance
Crime
Drama
Drama
Romance
Comedy
Comedy
Action
Comedy
Crime
Drama
Thriller
Comedy
Crime
Thriller
Crime
Drama
Horror
Mystery
Thriller
Action
Crime
Thriller
Drama
Sci-Fi
Drama
Romance
Drama
Children
Drama
Drama
Romance
Adventure
Drama
Fantasy
Mystery
Sci-Fi
Crime
Drama
Drama
Mystery
Sci-Fi
Thriller
Adventure
Romance
IMAX
Children
Drama
Drama
Romance
Crime
Drama
Documentary
IMAX
Children
Comedy
Comedy
Romance
Drama
Drama
War
Action
Crime
Drama
Drama
Action
Adventure
Fantasy
Comedy
Drama
Thriller
Drama
Romance
Mystery
Thriller
Animation
Children
Drama
Musical
Romance
Drama
Romance
Crime
Mystery
Thriller
Action
Drama
Thriller
Comedy
Drama
Romance
Adventure
Drama
Children
Comedy
Drama
Adventure
Chil

Drama
Romance
Fantasy
Horror
Drama
Comedy
Crime
Drama
Thriller
Adventure
Children
Action
Adventure
Animation
Horror
Sci-Fi
Action
Horror
Sci-Fi
Comedy
Drama
Romance
Drama
Thriller
Comedy
Drama
Animation
Children
Comedy
Drama
Comedy
Romance
Comedy
Drama
Drama
Romance
Action
Drama
Thriller
Drama
Comedy
Drama
Thriller
Crime
Drama
Mystery
Thriller
Drama
Drama
Romance
Adventure
Animation
Children
Fantasy
Musical
Romance
Drama
War
Comedy
Comedy
Comedy
Drama
Drama
Comedy
Romance
Comedy
Drama
Drama
Thriller
Adventure
Children
Comedy
Drama
Comedy
Comedy
Drama
Action
Crime
Drama
War
Action
Adventure
Mystery
Thriller
Comedy
Drama
Drama
Comedy
Horror
Mystery
Thriller
Action
Adventure
Fantasy
Drama
Romance
Comedy
Comedy
Comedy
Drama
Drama
Musical
Crime
Drama
Thriller
Drama
Adventure
Animation
Children
Fantasy
Musical
Thriller
Comedy
Comedy
Comedy
Drama
War
Drama
Romance
War
Action
Drama
Drama
Comedy
Sci-Fi
Drama
Adventure
Animation
Children
Comedy
Fantasy
Sci-Fi
Adventure
Comedy
Sci-Fi
Action
Drama

Drama
Fantasy
Crime
Film-Noir
Mystery
Thriller
Drama
Sci-Fi
Thriller
Action
Adventure
Drama
Western
Comedy
Horror
Sci-Fi
Comedy
Musical
War
Comedy
Romance
Horror
Adventure
Drama
Crime
Film-Noir
Thriller
Action
Comedy
Fantasy
Horror
Action
Adventure
Drama
War
Drama
War
Action
Drama
Mystery
Romance
Thriller
Comedy
Fantasy
Romance
Drama
Western
Crime
Thriller
War
Comedy
Drama
Comedy
Mystery
Thriller
Adventure
Comedy
Sci-Fi
Comedy
Crime
Drama
Drama
War
Comedy
Drama
Film-Noir
Action
Adventure
Animation
Sci-Fi
Action
Adventure
Fantasy
Drama
Comedy
Drama
Romance
Comedy
Fantasy
Comedy
Drama
Drama
Comedy
Drama
War
Animation
Children
Fantasy
Musical
Drama
Western
Crime
Film-Noir
Mystery
Comedy
Drama
Romance
Action
Adventure
Drama
Comedy
Documentary
Drama
Romance
Action
Adventure
Comedy
Drama
Drama
Drama
Drama
Romance
Comedy
Drama
Musical
Drama
War
Comedy
Drama
Drama
Sci-Fi
Children
Drama
Fantasy
Adventure
Drama
Action
Western
Drama
Romance
Adventure
Drama
Sci-Fi
Comedy
Romance
Comedy
Drama
Roman

Musical
Adventure
Animation
Children
Adventure
Animation
Children
Crime
Drama
Children
Sci-Fi
Adventure
Animation
Children
Fantasy
Musical
Romance
Adventure
Children
Fantasy
Action
Adventure
Sci-Fi
Children
Comedy
Animation
Children
Musical
Children
Drama
Fantasy
Mystery
Thriller
Children
Comedy
Adventure
Animation
Children
Musical
Comedy
Fantasy
Romance
Adventure
Drama
Animation
Children
Comedy
Musical
Adventure
Children
Fantasy
Western
Drama
Action
Adventure
Sci-Fi
Drama
War
Horror
Thriller
Comedy
Romance
Comedy
Comedy
Crime
Thriller
Comedy
Crime
Drama
Horror
Thriller
Drama
Action
Adventure
Fantasy
Adventure
Animation
Children
Fantasy
Drama
Sci-Fi
Thriller
Horror
Drama
Horror
Horror
Thriller
Horror
Thriller
Animation
Children
Comedy
Drama
Fantasy
Children
Comedy
Fantasy
Comedy
Drama
Romance
Action
Crime
Mystery
Thriller
Drama
Romance
Comedy
Documentary
Crime
Drama
Romance
Drama
Drama
Adventure
Comedy
Comedy
Fantasy
Sci-Fi
Adventure
Children
Musical
Comedy
Sci-Fi
Animation
Children
Ad

Thriller
Crime
Thriller
Comedy
Drama
Romance
War
Drama
Romance
Western
Crime
Drama
Western
Comedy
Action
Comedy
Drama
Comedy
Drama
Drama
Romance
Drama
Drama
Romance
Documentary
Comedy
Crime
Drama
Fantasy
Drama
Drama
Drama
Musical
Comedy
Romance
Adventure
Comedy
Romance
Comedy
Drama
Sci-Fi
Drama
Thriller
Drama
Film-Noir
Mystery
Romance
Musical
Romance
War
Drama
Romance
Drama
Romance
Action
Drama
War
Mystery
Comedy
Musical
Action
Adventure
Thriller
Action
Adventure
Thriller
Action
Adventure
Thriller
Adventure
Drama
Romance
Action
Western
Crime
Drama
Thriller
Children
Comedy
Drama
Drama
Action
Crime
Thriller
Drama
Drama
Action
Crime
Drama
Thriller
Drama
Comedy
Drama
Comedy
Romance
Crime
Drama
Drama
Action
Adventure
Drama
Drama
Thriller
Adventure
Comedy
Fantasy
Sci-Fi
Drama
Romance
Adventure
Drama
Drama
Fantasy
Musical
Drama
War
Comedy
Crime
Drama
Horror
Thriller
Comedy
Drama
Drama
Comedy
Drama
Comedy
Romance
Drama
Comedy
Drama
Documentary
Horror
Thriller
Thriller
Documentary
Action
Crime


...

Adventure
Western
(no genres listed)
Western
(no genres listed)
Action
Comedy
Comedy
Documentary
Documentary
(no genres listed)
Drama
Romance
War
Drama
Horror
Documentary
Horror
Comedy
Comedy
Comedy
Romance
Documentary
Drama
Fantasy
Action
Comedy
Drama
(no genres listed)
Children
Comedy
Drama
Adventure
Animation
Children
Fantasy
Horror
Sci-Fi
(no genres listed)
Animation
Children
Drama
Mystery
Romance
Action
Comedy
Thriller
Drama
Action
Adventure
Drama
Fantasy
Thriller
Thriller
Children
Drama
(no genres listed)
Crime
Drama
Mystery
Thriller
Action
Fantasy
Mystery
Drama
Drama
Romance
Drama
Comedy
Drama
Romance
Children
Drama
Fantasy
Comedy
Comedy
Drama
Sci-Fi
(no genres listed)
Comedy
Romance
Drama
Comedy
Drama
Romance
Comedy
Romance
(no genres listed)
(no genres listed)
Comedy
Drama
Action
Animation
Horror
Mystery
Comedy
Documentary
(no genres listed)
Drama
Drama
Thriller
Comedy
Drama
Drama
Sci-Fi
Comedy
Animation
Drama
Horror
Comedy
Drama
Action
Crime
Drama
Drama
Documentary
Drama
Horr

Documentary
Crime
Drama
Documentary
Mystery
Sci-Fi
Thriller
Comedy
Drama
(no genres listed)
Drama
Documentary
Documentary
Documentary
(no genres listed)
(no genres listed)
Comedy
Fantasy
Romance
Comedy
Drama
Comedy
(no genres listed)
Crime
Drama
Action
Drama
Crime
Drama
Horror
Documentary
Drama
Horror
Mystery
Thriller
Action
Adventure
Adventure
Drama
Drama
Thriller
(no genres listed)
Adventure
Children
Drama
Drama
(no genres listed)
Comedy
Drama
Romance
Thriller
Comedy
Drama
Mystery
Thriller
(no genres listed)
Drama
Documentary
(no genres listed)
Drama
Romance
Action
Drama
Thriller
Children
Comedy
Drama
Fantasy
Drama
Romance
Animation
Children
Fantasy
Documentary
Crime
Drama
Mystery
Drama
Action
Drama
Romance
Thriller
Drama
(no genres listed)
Action
Comedy
Crime
Drama
Mystery
Thriller
(no genres listed)
Documentary
Drama
Comedy
Horror
Documentary
Action
Crime
Thriller
Action
Horror
Thriller
Children
Drama
War
Documentary
(no genres listed)
(no genres listed)
Crime
Drama
Thriller
Childr

Documentary
Comedy
(no genres listed)
Drama
(no genres listed)
Comedy
Drama
Drama
Mystery
Thriller
Comedy
Drama
Horror
(no genres listed)
(no genres listed)
(no genres listed)
(no genres listed)
Drama
Romance
(no genres listed)
(no genres listed)
Drama
Romance
Action
Comedy
Drama
Romance
Drama
Romance
(no genres listed)
Drama
(no genres listed)
Comedy
Comedy
Drama
Romance
(no genres listed)
(no genres listed)
(no genres listed)
Action
Action
Crime
Drama
(no genres listed)
(no genres listed)
Comedy
Romance
Action
Sci-Fi
Action
Adventure
War
Documentary
Drama
Horror
(no genres listed)
(no genres listed)
(no genres listed)
(no genres listed)
Documentary
Horror
Comedy
Romance
(no genres listed)
Animation
Children
Comedy
Drama
Fantasy
Romance
Sci-Fi
Horror
Action
Adventure
Animation
Drama
Documentary
Drama
Action
Adventure
War
Crime
Drama
Thriller
Action
Crime
Drama
Mystery
Romance
Drama
(no genres listed)
(no genres listed)
Drama
Thriller
Documentary
War
Comedy
Drama
Documentary
Comedy
Rom

Children
Drama
Comedy
Romance
Comedy
Drama
Drama
Romance
Comedy
Fantasy
Animation
Children
Fantasy
(no genres listed)
Comedy
Comedy
Romance
Drama
Drama
Romance
Children
Fantasy
Comedy
Comedy
Romance
(no genres listed)
Action
Drama
Romance
Drama
Comedy
Drama
Romance
Comedy
Drama
Romance
Drama
Romance
Romance
War
Action
Comedy
Drama
Crime
Drama
Action
Adventure
Drama
Fantasy
Drama
Horror
Thriller
Comedy
Comedy
Crime
Drama
Comedy
(no genres listed)
Drama
Drama
Drama
Documentary
Children
Drama
(no genres listed)
Drama
Drama
Sci-Fi
Thriller
Drama
Documentary
Action
Animation
Fantasy
Thriller
Comedy
Horror
Crime
Drama
Drama
Drama
Romance
(no genres listed)
Action
Sci-Fi
Comedy
Action
Horror
Sci-Fi
(no genres listed)
Documentary
Drama
Drama
(no genres listed)
(no genres listed)
Drama
Drama
Action
Crime
Drama
Action
Comedy
Romance
Sci-Fi
Thriller
Comedy
(no genres listed)
(no genres listed)
Drama
Action
Animation
Romance
Comedy
Comedy
Comedy
Western
Documentary
Horror
Sci-Fi
Drama
Romance
Crim

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next, let's look at the ratings dataframe.

In [15]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

In [16]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<a id="ref3"></a>
# Content-Based recommendation system

Now, let's take a look at how to implement __Content-Based__ or __Item-Item recommendation systems__. This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the __userInput__. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [17]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


#### Add movieId to input user
With the input complete, let's extract the input movie's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movie's title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [19]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.

In [20]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.

In [21]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.

In [22]:
inputMovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [23]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.

Let's start by extracting the genre table from the original dataframe:

In [24]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
genreTable.shape

(34208, 20)

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.

In [26]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

In [27]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
5018      0.748252
26093     0.734266
27344     0.720280
148775    0.685315
6902      0.678322
dtype: float64

Now here's the recommendation table!

In [28]:
#The final recommendation table
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
664,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1824,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2902,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
4923,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
6793,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
8605,26093,"Wonderful World of the Brothers Grimm, The","[Adventure, Animation, Children, Comedy, Drama...",1962
8783,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
9825,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
11716,51632,Atlantis: Milo's Return,"[Action, Adventure, Animation, Children, Comed...",2003


### Advantages and Disadvantages of Content-Based Filtering

##### Advantages
* Learns user's preferences
* Highly personalized for the user

##### Disadvantages
* Doesn't take into account what others think of the item, so low quality item recommendations might happen
* Extracting data is not always intuitive
* Determining what characteristics of the item the user dislikes or likes is not always obvious

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p> 